In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster('local').setAppName('restaurant-review-average')
spark = SparkContext(conf=conf).getOrCreate()

24/12/05 10:32:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
import os

directory = os.path.join(os.getcwd(), 'data')
filename = 'restaurant_reviews.csv'
filepath = os.path.join(directory, filename)
lines = spark.textFile('file:///'+filepath.replace('\\', '/'))
lines

file:////home/lab11/src/data/restaurant_reviews.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [3]:
header = lines.first()

In [4]:
filtered_lines = lines.filter(lambda row : row !=header)

In [5]:
filtered_lines.take(5)

['0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,']

In [9]:
def parse(row):
    fields = row.split(",")
    category = fields[2]
    reviews = int(fields[3])
    return category, reviews

In [10]:
result1 = filtered_lines.map(parse).collect()

In [11]:
result2 = filtered_lines.map(parse).take(5)

In [12]:
result1

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [13]:
result2

[('중식', 125), ('중식', 235), ('분식', 32), ('분식', 534), ('일식', 223)]

In [14]:
cat_reviews = filtered_lines.map(parse).persist()
cat_reviews

PythonRDD[8] at RDD at PythonRDD.scala:53

In [16]:
result11 = cat_reviews.take(10)
result12 = cat_reviews.collect()

In [17]:
result11

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [18]:
result12

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [19]:
spark.stop()